# Chain-of-Verification GPT4 Template
Chain-of-Verification (CoVe) is a prompt engineering technique to bypass hallucinations. An LLM generates a baseline response to a user query, but this might contain errors. CoVe helps by creating a plan comprising of verification questions that are used to validate the information. This process results in more accurate answers than the initial response. The final answer is revised based on these validations. **[ Link to Paper](https://arxiv.org/pdf/2309.11495.pdf)**

**How to use template?**
1. Download `cove_template_config.json` [here](https://github.com/lastmile-ai/aiconfig/blob/main/cookbooks/Chain-of-Verification/cove_template_config.json).
2. Follow instructions in this notebook.

*Need a playground to test your prompts?*
Go to this [AI Workbook](https://lastmileai.dev/workbooks/clpa6nv2x00fdqp38oefmz29c) to experiment with your prompts/model params.

In [ ]:
# Install AIConfig package
!pip install python-aiconfig

In [ ]:
# Import required modules from AIConfig and other dependencies
import openai
import json
import pandas as pd
from google.colab import userdata
from aiconfig import AIConfigRuntime, CallbackManager, InferenceOptions

# Use your OpenAI Key
openai.api_key = userdata.get('openai_key')

In [ ]:
# Make sure to upload `cove_template_config.json`to Files in Colab Notebook
file_path = "cove_template_config.json"
config = AIConfigRuntime.load(file_path) # loads config
config.callback_manager = CallbackManager([])

inference_options = InferenceOptions() # setup streaming

## 1. Baseline Response
Prompt LLM with user question. The baseline response from the LLM might contain inaccuracies that we will want to verify.

**Prompt: Name 15 politicians who were born in New York City, New York.**

In [ ]:
# TODO: Update baseline_prompt but ensure it is structured in a way that outputs a list of entities where each can be verified.
baseline_prompt = "Name 15 politicians who were born in New York City, New York."
# baseline_prompt = "List 20 programming languages that were developed in the United States. Include the developer name in parantheses."

# Run baseline prompt to generate initial response which might contain errors
async def run_baseline_prompt(baseline_prompt):
    config.update_parameter("baseline_prompt", baseline_prompt)
    config.save()

    await config.run("baseline_response_gen", options=inference_options) # run baseline prompt
    return config.get_output_text("baseline_response_gen")

baseline_response = await run_baseline_prompt(baseline_prompt)

1. Theodore Roosevelt - 26th President of the United States
2. Franklin D. Roosevelt - 32nd President of the United States
3. Donald Trump - 45th President of the United States
4. Michael Bloomberg - Former Mayor of New York City
5. Rudy Giuliani - Former Mayor of New York City
6. Chuck Schumer - U.S. Senator from New York
7. Kirsten Gillibrand - U.S. Senator from New York
8. Alexandria Ocasio-Cortez - U.S. Representative from New York's 14th congressional district
9. Eliot Spitzer - Former Governor of New York
10. Andrew Cuomo - Former Governor of New York
11. Bill de Blasio - Former Mayor of New York City
12. Eric Adams - Current Mayor of New York City
13. Al Sharpton - Civil rights activist and former presidential candidate
14. Geraldine Ferraro - Former U.S. Representative and vice presidential candidate
15. Daniel Patrick Moynihan - Former U.S. Senator from New York.


## 2. Setup and Test Verification Question
Given both query and baseline response, generate a verification
question that could help to self-analyze if there are any mistakes in the original response. We will use one verification question here.

**Verification Prompt: Where was {{entity}} born?**

In [ ]:
# TODO: Update verification question that takes in entity as a parameter
verification_question = "Where was {{entity}} born?"
# verification_question =  "Where was this coding language developed: {{entity}}?"

# Run verification on a single entity from baseline response to test
async def run_single_verification(verification_question, entity):
    params = {"entity": entity}
    config.update_parameter("verification_question", verification_question)
    config.save()

    verification_completion = await config.run("verification", params, options=inference_options)
    return verification_completion

# TODO: Update with an entity from the baseline response
verification_completion = await run_single_verification(verification_question, "AL Sharpton")

Al Sharpton was born in Brooklyn, New York, USA.

## 3. Execute Verifications
Answer each verification question for each entity from the the baseline response. Save the verification results in a single string.

In [ ]:
# Extracts entity names from a given baseline response by processing each line with regex.
# TODO: Update regex if the format of the baseline response changes. (ex. not a numbered list)
def gen_entities_list(baseline_response):
  rows = baseline_response.split('\n')
  entities = []

  for row in rows:
      if not row.strip():
          continue
      entities.append(pd.Series(row).str.extract(r'(\d+\.\s)([^,]*)')[1].values[0])

  return entities

# Run verification question for each entity and concatenates returned verifications into a single string.
async def gen_verification_results(entities):
  verification_data = ""
  for n in entities:
      params = {
          "verification_question": verification_question,
          "entity": n
      }
      verification_completion = await config.run("verification", params, options=inference_options)
      single_verification_text = config.get_output_text("verification")
      verification_data += " " + single_verification_text
      print("\n")

  return verification_data


entities = gen_entities_list(baseline_response)
verification_data = await gen_verification_results(entities)

Theodore Roosevelt was born in New York City, New York, USA.

Franklin D. Roosevelt was born in Hyde Park, New York, USA.

Donald Trump was born in Queens, New York, USA.

Michael Bloomberg was born in Boston, Massachusetts, USA.

Rudy Giuliani was born in Brooklyn, New York, USA.

Chuck Schumer was born in Brooklyn, New York, USA.

Kirsten Gillibrand was born in Albany, New York, USA.

Alexandria Ocasio-Cortez was born in The Bronx, New York City, USA.

Eliot Spitzer was born in The Bronx, New York, USA.

Andrew Cuomo was born in Queens, New York, USA.

Bill de Blasio was born in Manhattan, New York, USA.

Eric Adams was born in Brownsville, Brooklyn, New York, USA.

Al Sharpton was born in Brooklyn, New York, USA.

Geraldine Ferraro was born in Newburgh, New York, USA.

Daniel Patrick Moynihan was born in Tulsa, Oklahoma, USA.



## 4. Generate Revised Response
Given the discovered inconsistencies (if any), generate a revised response incorporating the verification results.

In [ ]:
# Generated the revised response using verification data
params = {"verification_results": verification_data}
revised_response = await config.run("final_response_gen", params, options=inference_options)

### Revised Response 
1. Theodore Roosevelt - 26th President of the United States
2. Donald Trump - 45th President of the United States
3. Rudy Giuliani - Former Mayor of New York City
4. Chuck Schumer - U.S. Senator from New York
5. Alexandria Ocasio-Cortez - U.S. Representative from New York's 14th congressional district
6. Eliot Spitzer - Former Governor of New York
7. Andrew Cuomo - Former Governor of New York
8. Bill de Blasio - Former Mayor of New York City
9. Eric Adams - Current Mayor of New York City
10. Al Sharpton - Civil rights activist and former presidential candidate

### Failed Entities 
1. Franklin D. Roosevelt - Born in Hyde Park, New York, not New York City.
2. Michael Bloomberg - Born in Boston, Massachusetts, not New York City.
3. Kirsten Gillibrand - Born in Albany, New York, not New York City.
4. Geraldine Ferraro - Born in Newburgh, New York, not New York City.
5. Daniel Patrick Moynihan - Born in Tulsa, Oklahoma, not New York City.